In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

DEFAULT_COLS = [
    "electricity_pricing",
    "electricity_pricing_predicted_1",
    "electricity_pricing_predicted_2",
    "electricity_pricing_predicted_3",
]

def smard_to_citylearn(
    smard_in,
    out_csv,
    src_col=None,
    template_csv=None,   # falls du Länge+Spalten exakt übernehmen willst
    expected_len=None,   # alternativ: Länge direkt vorgeben
    columns=None         # alternativ: Spaltenliste direkt vorgeben
):
    smard_in = Path(smard_in)
    out_csv = Path(out_csv)

    # 1) Template/Erwartungen bestimmen
    cols_expected = None
    if template_csv is not None:
        tpl = pd.read_csv(template_csv)
        expected_len = len(tpl) if expected_len is None else expected_len
        cols_expected = list(tpl.columns)
    if columns is None and cols_expected is None:
        cols_expected = DEFAULT_COLS
    if columns is not None:
        cols_expected = list(columns)

    # 2) SMARD lesen (übliches SMARD-CSV: ; als Trennzeichen, , als Dezimal)
    df_raw = pd.read_csv(smard_in, sep=';', decimal=',')

    # 3) Preis-Spalte finden (oder vorgegeben)
    if src_col is None:
        candidates = [c for c in df_raw.columns if '€/MWh' in c]
        if not candidates:
            raise KeyError(
                "Keine Spalte mit '€/MWh' gefunden. Übergib src_col explizit."
            )
        src_col = candidates[0]

    # 4) €/MWh -> €/kWh
    p_mwh = pd.to_numeric(df_raw[src_col].replace('-', np.nan), errors='coerce')
    p = p_mwh / 1000.0

    # 5) 4 Spalten bauen (Predictions = Kopie; ggf. später mit shift ersetzen)
    df_out = pd.DataFrame({
        "electricity_pricing": p,
    })
    for i in (1, 2, 3):
        df_out[f"electricity_pricing_predicted_{i}"] = df_out["electricity_pricing"]

    # 6) Länge angleichen
    if expected_len is not None:
        if len(df_out) < expected_len:
            raise ValueError(
                f"SMARD hat {len(df_out)} Zeilen, erwartet sind {expected_len}. Zeitraum/Quelle angleichen."
            )
        df_out = df_out.iloc[:expected_len].ffill().bfill()

    # 7) Spaltenreihenfolge/-namen
    # Falls cols_expected genau die 4 Zielspalten (ggf. aus Template) enthält, setze Reihenfolge.
    if cols_expected is not None:
        # Wenn nur die Namen abweichen, aber 4 Spalten erwartet sind, umbenennen:
        if set(cols_expected) == set(DEFAULT_COLS):
            df_out = df_out[DEFAULT_COLS]
        elif set(cols_expected) == set(df_out.columns):
            df_out = df_out[cols_expected]  # nur Reihenfolge anpassen
        else:
            # Letzte Absicherung: Wenn cols_expected Länge 4 hat, mappen wir streng.
            if len(cols_expected) == 4:
                rename_map = dict(zip(DEFAULT_COLS, cols_expected))
                df_out = df_out.rename(columns=rename_map)[cols_expected]
            else:
                raise ValueError(
                    f"Unerwartete Spaltenvorgabe: {cols_expected}. Erwartet 4 Spalten."
                )

    # 8) Schreiben – direkt in die Datei, die deine schema.json nutzt
    df_out.to_csv(out_csv, index=False)
    print(f"Wrote {out_csv} with shape {df_out.shape} (src_col='{src_col}')")

if __name__ == "__main__":
    # BEISPIEL: Pfade anpassen
    data_dir = Path("Bachelorthesis_DQN_Agent/data/datasets/citylearn_challenge_2023_phase_3_1")

    SMARD_IN = data_dir / "pricing_germany_2023_june_to_august_raw.csv"           # deine SMARD-Rohdatei
    OUT_CSV  = data_dir / "pricing_germany_2023_june_to_august.csv"     # <<== DIESE Datei in schema.json eintragen
    TEMPLATE = data_dir / "pricing.csv"                     # optional: alte pricing.csv als Template

    smard_to_citylearn(
        smard_in=SMARD_IN,
        out_csv=OUT_CSV,
        template_csv=TEMPLATE,          # oder None, wenn du expected_len selbst setzt
        expected_len=None,              # z.B. 35040 für 15-minütige Daten über 1 Jahr
        columns=None,                   # sonst werden DEFAULT_COLS genommen
        src_col="Deutschland/Luxemburg [€/MWh] Originalauflösungen" # oder explizit: "Deutschland/Luxemburg [€/MWh] Originalauflösungen"
    )


Wrote Bachelorthesis_DQN_Agent/data/datasets/citylearn_challenge_2023_phase_3_1/pricing_germany_2023_june_to_august.csv with shape (2208, 4) (src_col='Deutschland/Luxemburg [€/MWh] Originalauflösungen')
